In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from torch.nn.utils.rnn import pad_sequence
from transformers import AutoTokenizer

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [ ]:
%pip install protobuf sentencepiece
%pip install x-transformers



   ------------- -------------------------- 2/6 [einops]
   ------------- -------------------------- 2/6 [einops]
   ------------- -------------------------- 2/6 [einops]
   -------------------- ------------------- 3/6 [loguru]
   -------------------- ------------------- 3/6 [loguru]
   -------------------------- ------------- 4/6 [einx]
   -------------------------- ------------- 4/6 [einx]
   -------------------------- ------------- 4/6 [einx]
   -------------------------- ------------- 4/6 [einx]
   -------------------------- ------------- 4/6 [einx]
   -------------------------- ------------- 4/6 [einx]
   -------------------------- ------------- 4/6 [einx]
   --------------------------------- ------ 5/6 [x-transformers]
   --------------------------------- ------ 5/6 [x-transformers]
   ---------------------------------------- 6/6 [x-transformers]

Note: you may need to restart the kernel to use updated packages.


In [5]:
tokenizer = AutoTokenizer.from_pretrained(
    "airesearch/wangchanberta-base-att-spm-uncased"
)
PAD_ID = tokenizer.pad_token_id
MAX_LEN = 256

In [24]:
df = pd.read_csv("C:\\Users\\User\\OneDrive\\Desktop\\seen_jupyter\\research\\archive\\prachatai_train.csv")
texts = df["body_text"].astype(str).tolist()
label_cols = [
    "politics", "human_rights", "quality_of_life", "international",
    "social", "environment", "economics", "culture", "labor",
    "national_security", "ict", "education"]

y = df[label_cols].values.astype(np.float32)

In [ ]:
def encode_texts(texts):
    enc = tokenizer(
        texts,
        truncation=True,
        max_length=MAX_LEN,
        padding=False,
        return_tensors=None
    )
    print("Encoding completed", enc)
    return enc["input_ids"]

encoded_texts = encode_texts(texts)
vocab_size = tokenizer.vocab_size
print("Vocabulary size:", vocab_size)

KeyboardInterrupt: 

: 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    encoded_texts, y, test_size=0.1, random_state=42)

In [10]:
class ThaiTextDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        seq = torch.tensor(self.X[idx], dtype=torch.long)
        return seq, self.y[idx]

def collate_fn(batch):
    seqs, labels = zip(*batch)
    padded = pad_sequence(seqs, batch_first=True, padding_value=PAD_ID)
    print("Padded batch :", padded,"Padded batch shape:" , padded.shape, "seqs length:", [len(s) for s in seqs], "labels:", labels)
    return padded.to(device), torch.stack(labels).to(device)

train_loader = DataLoader(
    ThaiTextDataset(X_train, y_train),
    batch_size=128,
    shuffle=True,
    collate_fn=collate_fn
)

test_loader = DataLoader(
    ThaiTextDataset(X_test, y_test),
    batch_size=128,
    collate_fn=collate_fn
)

train_loader

In [ ]:
from x_transformers import TransformerWrapper, Decoder

class XTransformerClassifier(nn.Module):
    def __init__(self, vocab_size, max_seq_len, output_dim, dim=512, depth=6, heads=8):
        super().__init__()
        self.transformer = TransformerWrapper(
            num_tokens=vocab_size,
            max_seq_len=max_seq_len,
            attn_layers=Decoder(
                dim=dim,
                depth=depth,
                heads=heads
            )
        )
        # Classification head
        self.classifier = nn.Linear(dim, output_dim)
        # self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Get transformer embeddings
        # TransformerWrapper returns logits, we need the hidden states
        emb = self.transformer(x, return_embeddings=True)
        
        # Pool over sequence dimension (mean pooling)
        mask = (x != PAD_ID).unsqueeze(-1)
        masked_emb = emb * mask
        pooled = masked_emb.sum(dim=1) / mask.sum(dim=1).clamp(min=1)
        
        # Classification
        out = self.classifier(pooled)
        return out

In [16]:
model = XTransformerClassifier(
    vocab_size=vocab_size,
    max_seq_len=MAX_LEN,
    output_dim=len(label_cols),
    dim=256,
    depth=4,
    heads=8
).to(device)

In [22]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 500
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        y_batch = y_batch.to(X_batch.device)
        preds = model(X_batch)
        loss = criterion(preds, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} | Loss: {total_loss:.4f}")

Padded batch : tensor([[    5, 11075,   109,  ...,    10, 13596,     6],
        [    5, 12556,     3,  ...,   300,  1740,     6],
        [    5,  2876,    12,  ...,  1075,  1755,     6],
        ...,
        [    5,    10,  1027,  ...,   506,   289,     6],
        [    5,  8716,    49,  ...,  2732,    10,     6],
        [    5,    10, 22288,  ...,  1654, 16292,     6]]) Padded batch shape: torch.Size([90, 256]) seqs length: [256, 256, 256, 256, 256, 256, 256, 256, 256, 133, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 215, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 212, 218, 256, 256, 256, 202, 256, 256, 256, 203, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 167, 256, 256, 256, 256, 256, 256, 256, 256, 256] labels: (tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [23]:
model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        preds = model(X_batch)
        preds = (preds > 0.5).int()
        y_true.append(y_batch.cpu().numpy())
        y_pred.append(preds.cpu().numpy())

y_true = np.vstack(y_true)
y_pred = np.vstack(y_pred)

print("F1 macro:", f1_score(y_true, y_pred, average="macro"))
print("F1 micro:", f1_score(y_true, y_pred, average="micro"))

Padded batch : tensor([[    5, 11261,    62,  ...,   938, 17004,     6],
        [    5, 13996,   853,  ...,    10,  2966,     6],
        [    5,  2876,    12,  ...,  9853,  1444,     6],
        ...,
        [    5,    10,  1428,  ...,   151, 14737,     6],
        [    5,    10, 18884,  ...,  9539,    45,     6],
        [    5,    10,   668,  ...,     1,     1,     1]]) Padded batch shape: torch.Size([10, 256]) seqs length: [256, 256, 256, 256, 256, 256, 256, 256, 256, 240] labels: (tensor([1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), tensor([1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]), tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), tensor([1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), tensor([0., 1., 0., 1., 0., 0., 0., 0., 0., 

c:\Users\User\OneDrive\Desktop\seen_jupyter\venv\lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [ ]:
def predict(text):
    enc = tokenizer(
        text,
        truncation=True,
        max_length=MAX_LEN,
        return_tensors="pt"
    )
    seq = enc["input_ids"].to(device)

    with torch.no_grad():
        probs = model(seq)[0].cpu().numpy()
        idx = np.argmax(probs)

    return label_cols[idx], float(probs[idx])

print(predict("รัฐบาลไทยประกาศนโยบายด้านสิ่งแวดล้อมใหม่"))
print(predict("แรงงานเรียกร้องสิทธิ์การทำงาน"))

('quality_of_life', 0.9999926090240479)
('politics', 0.016911478713154793)
